# College Football - Pull Data

The goal of this notebook is to pull in all the data from CollegeFootballData.com and export it to the data folder. 

#### Before running this notebook 

* Get custom API key emailed here: https://collegefootballdata.com/key
* Add it to a config/api_key.json file under the key api_key

Example of the structure of the config/api_key.json: 
```
{  
    "api_key": "InserttheAPIKeyhere"

}
```



##### Datasets extracted

1. **Recruiting information:** This data contians infromatioon about the high schooleers that were rectuited into college football between a certain timeframe. 

2. **Draft:** All the players that were drafted.

3. **Team:** All the data about the school they committed to (conference, coordiantes, etc).
 
4. **Team Momentum:** Get the two-year rolling win %, wins, and post season wins for the team. 


#### ----------------------------------

###### Helpful Tutorial
https://blog.collegefootballdata.com/introduction-to-cfb-analytics/

###### Actual Documentation
https://api.collegefootballdata.com/api/docs/?url=/api-docs.json

In [1]:
# Install the requirements
from IPython.display import clear_output

!pip install -r ../requirements.txt

clear_output()


In [2]:
# Uncomment and run line below if cfbd library isn't already installed
import cfbd
import numpy as np
import pandas as pd
import json

pd.set_option('display.max_columns', None)

## Set up api connection

In [3]:
# Running this code by itself won't work. You'll need your own API Key.
# See link above to have custom API link emailed and save that key as variable api_key.

# Load JSON data from file
with open('../config/api_key.json', 'r') as file:
    data = json.load(file)

# Get the value of 'api_key'
api_key = data.get('api_key')

In [4]:
def api_setup(api_key):

    """
    Configure the api. 
    Only input is the apikey which can be created from the link above.
    """
    import cfbd
    
    configuration = cfbd.Configuration()
    configuration.api_key['Authorization'] = api_key
    configuration.api_key_prefix['Authorization'] = 'Bearer'

    return cfbd.ApiClient(configuration)
    
api_config = api_setup(api_key)

## Define timeframe 

In [5]:
start_year_timeframe = 2008
end_year_timeframe = 2020 # The seniors in 2020 would've played all 4 years. Ie the Blake Corum year. 

## Get Datasource 1 - Player Recruiting Rankings

Get each football players ranking and origin information as they were recruited into college each year. 

In [6]:
def hs_recruits(start_year, end_year):
    
    """
    Two inputs: start_year and end_year (the ranges of years we want the recruiting data for - inclusive)
    
    1) Get each year as a json
    2) Convert to df
    3) Union each year's df together.
    """

    recruits_df_list = []

    for i in range(start_year, end_year + 1):

        # Connect to api for given year
        recr_api = cfbd.RecruitingApi(api_config)
        recruits = recr_api.get_recruiting_players(year = i)

        # Convert json to df
        df_recruits = pd.DataFrame.from_records([r.to_dict() for r in recruits])

        # Append dfs together to create list of dfs
        recruits_df_list.append(df_recruits)

    # Concatenate / union each year's df together
    df_recruits_final = pd.concat(recruits_df_list).reset_index()
    
    df_recruits_final['latitude'] = df_recruits_final.hometown_info.str['latitude']
    df_recruits_final['longitude'] = df_recruits_final.hometown_info.str['longitude']
    
    df_recruits_final.drop(columns = 'hometown_info', inplace = True)

    return df_recruits_final

df_recruits = hs_recruits(start_year_timeframe, end_year_timeframe)

In [7]:
df_recruits.groupby(by = 'year').agg({'year' : 'count'})

,year
year,
2008,2174
2009,2305
2010,2483
2011,2689
2012,3106
2013,3491
2014,3851
2015,3608
2016,4053


## Get Dataset 2 - Draft Data

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np 

In [9]:
def get_draft_data(year): 
    url = f"https://www.pro-football-reference.com/years/{year}/draft.htm"
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the table containing the draft data
        table = soup.find('table')

        # Use Pandas to read the HTML table into a DataFrame
        df = pd.read_html(str(table))[0].droplevel(0, axis=1)
        return df
    else:
        print("Failed to retrieve data. Status Code:", response.status_code)

def combine_draft_data(start_year, end_year): 
    
    years = np.arange(start_year, end_year, 1)
    print(f'Getting drafts for the following years: {years}. ')

    df_draft = pd.DataFrame() 
    for idx, year in enumerate(years): 
        if idx == 0: 
            df_draft = get_draft_data(years[0])
            df_draft['draft_year'] = year 
        else: 
            data_draft = get_draft_data(year)
            data_draft['draft_year'] = year 
            df_draft = pd.concat([df_draft, data_draft], axis=0)
    return df_draft

In [10]:
start_year_timeframe = 2010 
end_year_timeframe = 2024

In [11]:
# get draft data to include 2023
df_draft = combine_draft_data(start_year_timeframe, end_year_timeframe+1)

Getting drafts for the following years: [2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023
 2024]. 


/var/folders/wp/393dmtxd2jd8sj2z2tt7jpm40000gn/T/ipykernel_72666/3249126137.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0].droplevel(0, axis=1)
/var/folders/wp/393dmtxd2jd8sj2z2tt7jpm40000gn/T/ipykernel_72666/3249126137.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0].droplevel(0, axis=1)
/var/folders/wp/393dmtxd2jd8sj2z2tt7jpm40000gn/T/ipykernel_72666/3249126137.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0].droplevel(0, axis=1)
/var/folders/wp/393dmtxd2jd8sj2z2tt7jpm40000gn/T/ipykernel_72666/3249126137.

In [12]:
df_draft.head()

,Rnd,Pick,Tm,Player,Pos,Age,To,AP1,PB,St,wAV,DrAV,G,Cmp,Att,Yds,TD,Int,Att,Yds,TD,Rec,Yds,TD,Solo,Int,Sk,College/Univ,Unnamed: 28_level_1,draft_year
0,1,1,STL,Sam Bradford,QB,22,2018,0,0,5,44,25,83,1855,2967,19449,103,61,146,340,2,1,5,0,NaN,NaN,NaN,Oklahoma,College Stats,2010
1,1,2,DET,Ndamukong Suh,DT,23,2022,3,5,12,100,59,199,0,0,0,0,0,0,0,0,0,0,0,392,1,71.5,Nebraska,College Stats,2010
2,1,3,TAM,Gerald McCoy,DT,22,2021,1,6,10,69,65,140,0,0,0,0,0,0,0,0,0,0,0,235,NaN,59.5,Oklahoma,College Stats,2010
3,1,4,WAS,Trent Williams,T,22,2024,3,11,13,99,51,181,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,Oklahoma,College Stats,2010
4,1,5,KAN,Eric Berry,DB,21,2018,3,5,5,50,50,89,0,0,0,0,0,0,0,0,0,0,0,377,14,5.5,Tennessee,College Stats,2010


## Get Dataset 3 - College Team Data (conference, location, etc.)

In [13]:
def team_dataset():

    teams_api = cfbd.TeamsApi(api_config)
    teams = teams_api.get_fbs_teams()

    df_teams = pd.DataFrame.from_records([t.to_dict() for t in teams])
    
    # Extract coordinates from dict columns
    df_teams['latitude_school'] = df_teams['location'].apply(lambda x: x.get('latitude'))
    df_teams['longitude_school'] = df_teams['location'].apply(lambda x: x.get('longitude'))
    
    df_teams = df_teams[['id', 'school', 'conference', 'division', 'color', 'logos', 'latitude_school', 'longitude_school']]
    
    return df_teams

df_teams = team_dataset()

# Remove brackets around image url
df_teams['logos'] = df_teams['logos'].str.get(0)

In [14]:
df_teams.head()

,id,school,conference,division,color,logos,latitude_school,longitude_school
0,2005,Air Force,Mountain West,None,#004a7b,http://a.espncdn.com/i/teamlogos/ncaa/500/2005...,38.996970,-104.843616
1,2006,Akron,Mid-American,None,#00285e,http://a.espncdn.com/i/teamlogos/ncaa/500/2006...,41.072553,-81.508341
2,333,Alabama,SEC,None,#9e1632,http://a.espncdn.com/i/teamlogos/ncaa/500/333.png,33.208275,-87.550384
3,2026,App State,Sun Belt,East,#ffcc00,http://a.espncdn.com/i/teamlogos/ncaa/500/2026...,36.211427,-81.685428
4,12,Arizona,Big 12,None,#0c234b,http://a.espncdn.com/i/teamlogos/ncaa/500/12.png,32.228805,-110.948868


## Get Dataset 4 - Momentum / prominence of each team at point of committment
- At the point of committment, how many wins, bowl appearances, winning % has the team had in the 2 years running up to the recruits committment.
- This is essentially a check to see how prominent the program was leading up to the recruit's committment

In [15]:
start_year_games = 2003
end_year_games = 2020

In [16]:
def games_non_transformed(start_year, final_year):
    
    
    """
    Connect to the games api and get every post and regular season game over a given time frame
    
    1) Beginning with post season, iterate over every year in given range.
    2) Union each year together
    3) Repeat for regular season
    4) Union post and regular season dfs together
    """

    # Connect to games api
    games_api = cfbd.GamesApi(api_config)

    # Post Season Games
    postseason_games = []
    for i in range(start_year, final_year + 1):

        games = games_api.get_games(year=i, season_type = 'postseason')
        df_games_post_i = pd.DataFrame.from_records([g.to_dict() for g in games])
        postseason_games.append(df_games_post_i)

    postseason_games_df = pd.concat(postseason_games)

    # Regular Season Games
    regseason_games = []
    for i in range(start_year, final_year + 1):

        games = games_api.get_games(year=i, season_type = 'regular')
        df_games_reg_i = pd.DataFrame.from_records([g.to_dict() for g in games])
        regseason_games.append(df_games_reg_i)

    regseason_games_df = pd.concat(regseason_games)

    # Union post and regular season
    return pd.concat([regseason_games_df, postseason_games_df])

df_games = games_non_transformed(start_year_timeframe, end_year_timeframe)

/var/folders/wp/393dmtxd2jd8sj2z2tt7jpm40000gn/T/ipykernel_72666/60550415.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  postseason_games_df = pd.concat(postseason_games)
/var/folders/wp/393dmtxd2jd8sj2z2tt7jpm40000gn/T/ipykernel_72666/60550415.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  regseason_games_df = pd.concat(regseason_games)


In [17]:
def games_manipulation(df_games):
    
    """
    The function takes in the output of the previous function games_non_transformed().
    It converts the grain of the data. Before, the grain was 1 row per game.
    Now, each game has two rows: one from the perspective of each team.
    For example, when Ohio lost to Michigan 42-27, Michigan will a win by 15 points, and OSU will show a loss by -15 points.
    
    1) First identifies every team that played at least 1 home game.
    2) Loops over every team.
    3) Converts a bunch of data points so that the numbers are referenced from the perspective of the team of interst (also called main_team)
    """


    df_seasons = []
    teams_list = list(df_games['home_team'].unique()[1:])

    # Loop over every team that played a game over the time frame specified in games_non_transformed()
    for team in teams_list:

        # find every home / away game for team of interst
        df_home = df_games[df_games['home_team'] == team]
        df_away = df_games[df_games['away_team'] == team]

        # Combine home and away games into 1 table. 
        df_season_i = pd.concat([df_home, df_away])

        # Add column specifying what team that row of data pertains to
        df_season_i['main_team'] = team

        ############
        # Adjust key columns so they represent our team of interest

        df_season_i['home_game_flag'] = np.where(df_season_i['home_team'] == team, 1, 0)

        df_season_i['team_id'] = np.where(df_season_i['home_team'] == team, df_season_i['home_id'], df_season_i['away_id'])
        df_season_i['opposing_team_id'] = np.where(df_season_i['home_team'] == team, df_season_i['away_id'], df_season_i['home_id'])

        df_season_i['team_conference'] = np.where(df_season_i['home_team'] == team, df_season_i['home_conference'], df_season_i['away_conference'])
        df_season_i['opposing_conference'] = np.where(df_season_i['home_team'] == team, df_season_i['away_conference'], df_season_i['home_conference'])

        df_season_i['points_for'] = np.where(df_season_i['home_team'] == team, df_season_i['home_points'], df_season_i['away_points'])
        df_season_i['points_against'] = np.where(df_season_i['home_team']== team, df_season_i['away_points'], df_season_i['home_points'])

        df_season_i['point_differential'] = df_season_i['points_for'] - df_season_i['points_against']

        df_season_i['team_line_scores']  = np.where(df_season_i['home_team'] == team, df_season_i['home_line_scores'], df_season_i['away_line_scores'])
        df_season_i['opposing_line_scores']  = np.where(df_season_i['home_team'] == team, df_season_i['away_line_scores'], df_season_i['home_line_scores'])

        df_season_i['team_pregame_elo']  = np.where(df_season_i['home_team'] == team, df_season_i['home_pregame_elo'], df_season_i['away_pregame_elo'])
        df_season_i['team_postgame_elo']  = np.where(df_season_i['home_team'] == team, df_season_i['home_postgame_elo'], df_season_i['away_postgame_elo'])

        df_season_i['opponent_pregame_elo'] = np.where(df_season_i['home_team'] != team, df_season_i['home_pregame_elo'], df_season_i['away_pregame_elo'])
        df_season_i['opponent_postgame_elo'] = np.where(df_season_i['home_team'] != team, df_season_i['home_postgame_elo'], df_season_i['away_postgame_elo'])

        df_season_i['win_flag'] = np.where(df_season_i['point_differential'] > 0, 1, 0)
        
#         a = df_season_i.sort_values('start_date', ascending=True) \
#                        .groupby(['main_team', 'season']) \
#                        .cumcount() + 1
        
#         df_season_i['game_that_season'] = list(a)
        
        
        ############

        df_seasons.append(df_season_i)
        
    data = pd.concat(df_seasons)
    data = data.drop(columns = ['home_id', 'home_team', 'home_conference', 'home_division', 'home_points', 'home_line_scores', 'home_post_win_prob',
                                'home_pregame_elo', 'home_postgame_elo', 'away_id', 'away_team', 'away_conference', 'away_division', 'away_points',
                                'away_line_scores', 'away_post_win_prob', 'away_pregame_elo', 'away_postgame_elo'], axis = 1)
    
    # Field that counts what game (ie the 15th game, 3rd game, etc)
    data = data.reset_index()
    data['game_that_season'] = data.sort_values(['season', 'start_date', 'team_id'], ascending=True) \
                                        .groupby(['team_id', 'season']) \
                                        .cumcount() + 1
    
    
    
    return data
    
df_manipulated_games = games_manipulation(df_games)

In [18]:
import warnings
warnings.filterwarnings('ignore')

# aggregate to the season grain, and summarize game results
df_games_st = df_manipulated_games[['id', 'main_team', 'season', 'season_type', 'win_flag', 'point_differential']]
df_games_st['post_season_win_flag'] = np.where((df_games_st['season_type'] == 'postseason') & (df_games_st['win_flag'] == 1), 1, 0)
df_season = df_games_st.groupby(by = ['season', 'main_team']).agg({'win_flag' : 'sum',
                                                                   'point_differential': 'sum',
                                                                   'post_season_win_flag': 'sum',
                                                                   'id': 'count'}).reset_index()

df_season.columns = ['season', 'team', 'wins', 'point_diff', 'post_season_wins', 'games_played']

# lag over the team to get their previous year's data
df_season[['prev_wins', 'prev_point_diff', 'prev_post_season_wins', 'prev_games_played']] = (df_season.sort_values(by=['season', 'team'], ascending=True)\
                                                                                                      .groupby(['team'])[['wins', 'point_diff', 'post_season_wins', 'games_played']]\
                                                                                                      .shift(1))
# lag over the team to get data from two years ago
df_season[['2years_wins', '2years_point_diff', '2years_post_season_wins', '2years_games_played']] = (df_season.sort_values(by=['season', 'team'], ascending=True)\
                                                                                                      .groupby(['team'])[['wins', 'point_diff', 'post_season_wins', 'games_played']]\
                                                                                                      .shift(2))
# combine lagged data into one column:
df_season['wins_rolling_2year'] = df_season['prev_wins'] + df_season['2years_wins']
df_season['games_played_rolling_2year'] = df_season['prev_games_played'] + df_season['2years_games_played']
df_season['post_season_wins_rolling_2year'] = df_season['prev_post_season_wins'] + df_season['2years_post_season_wins']
df_season['point_diff_rolling_2year'] = df_season['prev_point_diff'] + df_season['2years_point_diff']
df_season['win_pct_rolling_2year'] = df_season['wins_rolling_2year'] / df_season['games_played_rolling_2year']

# drop the lagged data cols now that they're combined into new columns"
df_season = df_season.drop(columns = ['2years_wins', '2years_point_diff', '2years_post_season_wins', '2years_games_played', 
                                      'prev_wins', 'prev_point_diff', 'prev_post_season_wins', 'prev_games_played',
                                      'wins', 'point_diff', 'post_season_wins', 'games_played'])

df_season.sort_values(by = ['team', 'season']).head()

,season,team,wins_rolling_2year,games_played_rolling_2year,post_season_wins_rolling_2year,point_diff_rolling_2year,win_pct_rolling_2year
867,2013,Abilene Christian,NaN,NaN,NaN,NaN,NaN
1179,2014,Abilene Christian,NaN,NaN,NaN,NaN,NaN
1482,2015,Abilene Christian,11.0,22.0,0.0,265.0,0.500000
1779,2016,Abilene Christian,8.0,21.0,0.0,-12.0,0.380952
2076,2017,Abilene Christian,5.0,21.0,0.0,-180.0,0.238095


#### Sanity checked on rolling two year data:

In [19]:
df_season[df_season['team'] == 'Kennesaw State']

,season,team,wins_rolling_2year,games_played_rolling_2year,post_season_wins_rolling_2year,point_diff_rolling_2year,win_pct_rolling_2year
1595,2015,Kennesaw State,NaN,NaN,NaN,NaN,NaN
1890,2016,Kennesaw State,NaN,NaN,NaN,NaN,NaN
2183,2017,Kennesaw State,14.0,22.0,0.0,274.0,0.636364
2480,2018,Kennesaw State,20.0,25.0,0.0,393.0,0.800000
2777,2019,Kennesaw State,23.0,27.0,0.0,569.0,0.851852
3290,2021,Kennesaw State,22.0,27.0,0.0,633.0,0.814815
3846,2022,Kennesaw State,22.0,27.0,0.0,442.0,0.814815
4533,2023,Kennesaw State,16.0,24.0,0.0,91.0,0.666667
5226,2024,Kennesaw State,8.0,20.0,0.0,-9.0,0.400000


In [20]:
# Spot check original / unlagged results for a single team
df_games_st = df_manipulated_games[['id', 'main_team', 'season', 'season_type', 'win_flag', 'point_differential']]
df_games_st['post_season_win_flag'] = np.where((df_games_st['season_type'] == 'postseason') & (df_games_st['win_flag'] == 1), 1, 0)
df_games_st = df_games_st.groupby(by = ['season', 'main_team']).agg({'win_flag' : 'sum',
                                                                   'point_differential': 'sum',
                                                                   'post_season_win_flag': 'sum',
                                                                   'id': 'count'}).reset_index()

df_games_st[df_games_st['main_team'] == 'Michigan']

,season,main_team,win_flag,point_differential,post_season_win_flag,id
143,2010,Michigan,7,-32.0,0,13
436,2011,Michigan,11,207.0,1,13
718,2012,Michigan,8,130.0,0,13
1013,2013,Michigan,7,70.0,0,13
1324,2014,Michigan,5,-18.0,0,12
1622,2015,Michigan,10,195.0,1,13
1919,2016,Michigan,10,341.0,0,13
2214,2017,Michigan,8,83.0,0,13
2510,2018,Michigan,10,205.0,0,13
2808,2019,Michigan,9,143.0,0,13


In [21]:
df_season[df_season['team'] == 'Michigan']

,season,team,wins_rolling_2year,games_played_rolling_2year,post_season_wins_rolling_2year,point_diff_rolling_2year,win_pct_rolling_2year
143,2010,Michigan,NaN,NaN,NaN,NaN,NaN
436,2011,Michigan,NaN,NaN,NaN,NaN,NaN
718,2012,Michigan,18.0,26.0,1.0,175.0,0.692308
1013,2013,Michigan,19.0,26.0,1.0,337.0,0.730769
1324,2014,Michigan,15.0,26.0,0.0,200.0,0.576923
1622,2015,Michigan,12.0,25.0,0.0,52.0,0.480000
1919,2016,Michigan,15.0,25.0,1.0,177.0,0.600000
2214,2017,Michigan,20.0,26.0,1.0,536.0,0.769231
2510,2018,Michigan,18.0,26.0,0.0,424.0,0.692308
2808,2019,Michigan,18.0,26.0,0.0,288.0,0.692308


In [22]:
df_season.head()

,season,team,wins_rolling_2year,games_played_rolling_2year,post_season_wins_rolling_2year,point_diff_rolling_2year,win_pct_rolling_2year
0,2010,Adams State,NaN,NaN,NaN,NaN,NaN
1,2010,Air Force,NaN,NaN,NaN,NaN,NaN
2,2010,Akron,NaN,NaN,NaN,NaN,NaN
3,2010,Alabama,NaN,NaN,NaN,NaN,NaN
4,2010,Alabama A&M,NaN,NaN,NaN,NaN,NaN


In [23]:
df_team_season = pd.merge(left = df_teams, right = df_season, how = 'left', left_on = 'school', right_on = 'team')
df_team_season = df_team_season[['school', 'division', 'latitude_school', 'longitude_school', 'season', 'wins_rolling_2year', 'games_played_rolling_2year', 'post_season_wins_rolling_2year','point_diff_rolling_2year', 'win_pct_rolling_2year']]

In [24]:
df_team_season

,school,division,latitude_school,longitude_school,season,wins_rolling_2year,games_played_rolling_2year,post_season_wins_rolling_2year,point_diff_rolling_2year,win_pct_rolling_2year
0,Air Force,None,38.99697,-104.843616,2010.0,NaN,NaN,NaN,NaN,NaN
1,Air Force,None,38.99697,-104.843616,2011.0,NaN,NaN,NaN,NaN,NaN
2,Air Force,None,38.99697,-104.843616,2012.0,16.0,26.0,1.0,212.0,0.615385
3,Air Force,None,38.99697,-104.843616,2013.0,13.0,26.0,0.0,64.0,0.500000
4,Air Force,None,38.99697,-104.843616,2014.0,8.0,25.0,0.0,-205.0,0.320000
...,...,...,...,...,...,...,...,...,...,...
1975,Wyoming,None,41.31161,-105.568138,2020.0,14.0,25.0,1.0,83.0,0.560000
1976,Wyoming,None,41.31161,-105.568138,2021.0,10.0,19.0,1.0,132.0,0.526316
1977,Wyoming,None,41.31161,-105.568138,2022.0,9.0,19.0,1.0,55.0,0.473684
1978,Wyoming,None,41.31161,-105.568138,2023.0,14.0,26.0,1.0,-13.0,0.538462


### Save all to CSV in the 'data' file

In [25]:
df_draft.to_csv('../data/draft_2010_2024.csv', index = False)
print('df_draft: ' + str(df_draft.shape))

df_recruits.to_csv('../data/recruits_2008_2020.csv', index = False)
print('df_recruits: ' + str(df_recruits.shape))

df_teams.to_csv('../data/teams.csv', index = False)
print('df_teams: ' + str(df_teams.shape))

df_team_season.to_csv('../data/team_season.csv', index = False)
print('df_team_season: ' + str(df_team_season.shape))

df_draft: (3926, 30)
df_recruits: (44541, 19)
df_teams: (134, 8)
df_team_season: (1980, 10)
